In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests

from pvlib import pvsystem, modelchain, location, irradiance
from pvlib.solarposition import get_solarposition

In [5]:
def get_hourly_weather_data_for_pvlib(stations, start_date, end_date, timezone = 'UTC'):
    '''
    Function to get hourly weather variables T (temperature) and
    Q (global radiation) from KNMI 
    
    Args: 
        stations   (str): NMI-stations separated by ':' 
        start_date (str): start date, format yyyymmdd
        end_date   (str): end date (included), format yyyymmdd
        timezone   (str, optional): timezone

    Returns:
        df: DataFrame with DateTime-index, columns T (temp), Q (global radiation) 
    '''

    url = 'https://www.daggegevens.knmi.nl/klimatologie/uurgegevens'

    data = {
        'start': start_date,
        'end': end_date,
        'vars': 'Q:P:T',
        'stns': stations,
        'fmt': 'json'
        }

    response = requests.post(url, data = data)    
    weather_df = pd.DataFrame(response.json())
    print(weather_df)
    # correct units
    weather_df['T'] = weather_df['T'] / 10          # is in 0.1 degrees C, to degrees C
    weather_df['Q'] = weather_df['Q'] * (1 / 0.36)  # is in J/m2, to W / m2
    weather_df['P'] = weather_df['P'] * 10          # is in 0.1 hPa, to Pa
    
    # create date_time index, convert timezone
    weather_df['hour'] = weather_df['hour'] - 1     # is from 1-24, to 0-23
    weather_df['date_time'] = pd.to_datetime(weather_df['date']) + pd.to_timedelta(weather_df['hour'].astype(int), unit='h')
    weather_df.index = weather_df.date_time
    weather_df = weather_df.drop(['date', 'hour', 'date_time'], axis = 1)
    weather_df.index = weather_df.index.tz_convert(timezone)

    # shift date_time by 30 minutes, 'average time' during that hour
    weather_df.index = weather_df.index.shift(freq="30min")

    return weather_df

In [11]:
# Whole year of 2023
start_date = '20240130'
end_date = '20241230'

timezone = 'Europe/Amsterdam'

# Eindhoven KNMI STATION
station = '370'
lat = 51.449772459909
lon = 5.3770039280214

# Function get_hourly_weather_data_for_pvlib defined in full code overview below
weather_df = get_hourly_weather_data_for_pvlib(station, start_date, end_date, timezone)

Empty DataFrame
Columns: []
Index: []


KeyError: 'T'

In [ ]:
# Get solar position for the dates / times
solpos_df = get_solarposition(
    weather_df.index, latitude = lat,
    longitude = lon, altitude = 0,
    temperature = weather_df['T'])
solpos_df.index = weather_df.index

# Method 'Erbs' to go from GHI to DNI and DHI
irradiance_df = irradiance.erbs(weather_df['Q'], solpos_df['zenith'], weather_df.index)
irradiance_df['ghi'] = weather_df['Q']

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    print(irradiance_df)

In [ ]:
from netCDF4 import Dataset

# Open the NetCDF file
nc_file = "C:/Users/20193362/Desktop/STD___OPER_P___OBS_____L2.nc"
ds = Dataset(nc_file, mode='r')  # 'r' is for read-only mode


names = ds.variables['name']
station_names_list = [str(name) for name in names[:]]

# Target station name
target_name = "EINDHOVEN AP"

# Find the index
station_index = station_names_list.index(target_name)

name = ds.variables['name'][station_index]
latitude = ds.variables['lat'][station_index]
longitude = ds.variables['lon'][station_index]
print(f"Coordinates for station {name}: Latitude {latitude}, Longitude {longitude}")

# Close the data
ds.close()